<a href="https://colab.research.google.com/github/prashantpandya000/21_days_prashant_pandya/blob/main/reinforcement_learning_day1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Install Dependencies



In [3]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

     |████████████████████████████████| 320.4MB 48kB/s 
     |████████████████████████████████| 460kB 42.5MB/s 
     |████████████████████████████████| 20.1MB 11.2MB/s 
     |████████████████████████████████| 2.9MB 45.8MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installation: tensorflow

     |████████████████████████████████| 61kB 5.7MB/s 


##1. Test Random Environment with OpenAI Gym

In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [10]:

class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -3 
        # Reduce shower length by 1 second
        self.shower_length -= 3 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =3 
        else: 
            reward = -3 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [11]:
env = ShowerEnv()

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [12]:
env.observation_space.sample()

array([86.29862], dtype=float32)

In [13]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-48
Episode:2 Score:-60
Episode:3 Score:-54
Episode:4 Score:-60
Episode:5 Score:-54
Episode:6 Score:-60
Episode:7 Score:-60
Episode:8 Score:-60
Episode:9 Score:-60
Episode:10 Score:-48


##2. Create a Deep Learning Model with Keras

In [28]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [29]:
states = env.observation_space.shape
actions = env.action_space.n

In [30]:
actions

3

In [31]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [32]:
model = build_model(states, actions)

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________



#3. Build Agent with Keras-RL

In [34]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [35]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [36]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 4:02:13 - reward: -3.0000

/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 88s 9ms/step - reward: -2.8152
500 episodes - episode_reward: -56.304 [-60.000, -42.000] - loss: 1.840 - mae: 15.418 - mean_q: -21.817

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 90s 9ms/step - reward: -2.7756
500 episodes - episode_reward: -55.512 [-60.000, -42.000] - loss: 1.212 - mae: 17.615 - mean_q: -25.204

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 88s 9ms/step - reward: -2.7588
500 episodes - episode_reward: -55.176 [-60.000, -42.000] - loss: 1.077 - mae: 17.792 - mean_q: -25.431

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 89s 9ms/step - reward: -2.7510
500 episodes - episode_reward: -55.020 [-60.000, -42.000] - loss: 1.025 - mae: 17.823 - mean_q: -25.445

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 89s 9ms/step - reward: -2.7078
done, took 444.030 seconds


In [37]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -42.000, steps: 20
Episode 2: reward: -42.000, steps: 20
Episode 3: reward: -42.000, steps: 20
Episode 4: reward: -54.000, steps: 20
Episode 5: reward: -60.000, steps: 20
Episode 6: reward: -60.000, steps: 20
Episode 7: reward: -60.000, steps: 20
Episode 8: reward: -42.000, steps: 20
Episode 9: reward: -42.000, steps: 20
Episode 10: reward: -42.000, steps: 20
Episode 11: reward: -42.000, steps: 20
Episode 12: reward: -60.000, steps: 20
Episode 13: reward: -42.000, steps: 20
Episode 14: reward: -42.000, steps: 20
Episode 15: reward: -60.000, steps: 20
Episode 16: reward: -48.000, steps: 20
Episode 17: reward: -60.000, steps: 20
Episode 18: reward: -60.000, steps: 20
Episode 19: reward: -42.000, steps: 20
Episode 20: reward: -42.000, steps: 20
Episode 21: reward: -60.000, steps: 20
Episode 22: reward: -60.000, steps: 20
Episode 23: reward: -42.000, steps: 20
Episode 24: reward: -42.000, steps: 20
Episode 25: reward: -60.000, steps: 20
Episo

#4. Reloading Agent from Memory

In [48]:

dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [49]:
del model
del dqn
del env

In [43]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [46]:
env=dqn.load_weights('dqn_weights.h5f')